# Importing required libraries 

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sn

# Load the data

In [14]:

exp_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/eu_health_expending.csv", compression='gzip')
life_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/life-expectancy.csv", compression='gzip')
mort_df = pd.read_csv("C:/Users/39351/github/Eu_healthcare_spending_analysis/data/raw/mortality_rate.csv", compression='gzip')


# Cleaning the Data

In [ ]:

exp_df.head()
exp_df.columns, exp_df.shape
exp_df['Classification of health care financing schemes - SHA 2011'].value_counts()

# remove unnecessary columns
exp_clean = exp_df[['geo',
        'Geopolitical entity (reporting)', 'TIME_PERIOD', 'unit', 'icha11_hf','Classification of health care financing schemes - SHA 2011', 'OBS_VALUE']]


# renaming the columns
exp_clean.rename(columns={
    'geo': 'Country Code',
    'Geopolitical entity (reporting)': 'Country Name', 
    'TIME_PERIOD': 'year', 
    'unit': 'unit', 
    'icha11_hf': 'financing_code',
    'Classification of health care financing schemes - SHA 2011': 'category', 
    'OBS_VALUE': 'spending'
}, inplace=True)




In [ ]:
# life_df.head()
# life_df.columns, life_df.shape

# mort_df.head()
# mort_df.columns, mort_df.shape